In [1]:
#from datetime import datetime
#from dateutil.relativedelta import relativedelta
#import os

#import pandas as pd
import requests

#print('import done')

In [59]:
# get the headers of the taxi data
url = 'https://futar.bkk.hu/api/query/v1/ws/otp/api/where/bicycle-rental.json?key=bd29005b-80cb-499b-968b-c7be1e6b3d37&version=3&appVersion=1&includeReferences=false'

response = requests.get(url)
data = response.json()
data

{'currentTime': 1705001643177,
 'version': 3,
 'status': 'OK',
 'code': 200,
 'text': 'OK',
 'data': {'list': [{'id': '42711896',
    'lat': 47.51032,
    'lon': 19.028615,
    'name': 'Millenáris',
    'code': '0213',
    'type': 'stele',
    'bikes': 1,
    'spaces': 999,
    'style': {'icon': {'name': 'vehicle-rental/mol-bubi'}}},
   {'id': '344201519',
    'lat': 47.479564,
    'lon': 19.050729,
    'name': '860227',
    'code': 'BIKE',
    'bikes': 1,
    'spaces': 999,
    'style': {'icon': {'name': 'vehicle-rental/mol-bubi-floating'}}},
   {'id': '344214154',
    'lat': 47.490413,
    'lon': 19.066131,
    'name': '861120',
    'code': 'BIKE',
    'bikes': 1,
    'spaces': 999,
    'style': {'icon': {'name': 'vehicle-rental/mol-bubi-floating'}}},
   {'id': '42990800',
    'lat': 47.475277,
    'lon': 19.061092,
    'name': 'Pázmány Péter sétány - Északi tömb',
    'code': '1118',
    'type': 'stele',
    'bikes': 10,
    'spaces': 999,
    'style': {'icon': {'name': 'vehicle-ren

In [60]:
extracteddata = []
atl = 0
for mylist in data["data"]["list"]:
    if mylist["code"] != "BIKES" and mylist["spaces"] != "0":
        extracteddata.append(mylist["bikes"])
for i in range(len(extracteddata)):
    atl += extracteddata[i]
atl /= len(extracteddata)
print(f"{str(atl)[0:4]}")
    

7.24
